<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning - Load Historica data to emulate a production system

### Contents

- [1.0 Install Python Packages](#setup)
- [2.0 Configure Credentials](#credentials)
- [3.0 OpenScale configuration](#openscale)
- [4.0 Get Subscriptions](#subscription)
- [5.0 Historical Data](#historical)

# 1.0 Install Python Packages

In [2]:
!rm -rf /home/spark/shared/user-libs/python3.6*

!pip install --upgrade ibm-ai-openscale==2.2.1 --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client-V4==1.0.110 | tail -n 1
!pip install --upgrade pyspark==2.3 | tail -n 1

ERROR: Could not install packages due to an EnvironmentError: [Errno 30] Read-only file system: 'INSTALLER'

    Uninstalling ibm-ai-openscale-2.1.21:


### Action: restart the kernel!

# 2.0 Configure credentials <a name="credentials"></a>



In [ ]:
import warnings
warnings.filterwarnings('ignore')

The url for `WOS_CREDENTIALS` is the url of the CP4D cluster, i.e. `https://zen-cpd-zen.apps.com`. `username` and `password` must be ones that have cluster admin privileges.

In [1]:
WOS_CREDENTIALS = {
    "url": "https://zen1-cpd-zen1.aida-cpd3-dal10-b3c32x128-f2c6cdc6801be85fd188b09d006f13e3-0001.us-south.containers.appdomain.cloud",
    "username": "scottda-noadmin",
    "password": "<redacted>"
}

In [2]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='3.0.0'

In [3]:
%store -r MODEL_NAME
%store -r DEPLOYMENT_NAME
%store -r DEFAULT_SPACE
%store -r model_uid
%store -r binding_uid
%store -r deployment_uid

no stored variable or alias MODEL_NAME
no stored variable or alias DEPLOYMENT_NAME
no stored variable or alias DEFAULT_SPACE


In [4]:
MODEL_NAME = "sda-model-8-3-2020"
DEPLOYMENT_NAME = "scottda-model-deployment-8-3-2020"
DEFAULT_SPACE = "3f2c0a53-e1bb-4e09-957d-d7b84c16a56f"

# 3.0 Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and configure OpenScale

In [5]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [6]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [7]:
ai_client = APIClient4ICP(WOS_CREDENTIALS)
ai_client.version

'2.1.21'

# 4.0 Get Subscription <a name="subscription"></a>

In [8]:
subscription = None

if subscription is None:
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            print("Found existing subscription.")
            subscription = ai_client.data_mart.subscriptions.get(sub)
            subscription_uid = sub
            print("subscription_uid: " + sub)
if subscription is None:
    print("No subscription found. Please run openscale-initial-setup.ipynb to configure.")

Found existing subscription.
subscription_uid: f978ce05-e4f6-42b0-81b2-be06884cd358


### Set Deployment UID

In [9]:
wml_client.set.default_space(DEFAULT_SPACE)

'SUCCESS'

In [10]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    print(deployment['entity']['name'])
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break
        
print(deployment_uid)

WOS-INTERNAL-e11cebdc-d53f-4812-b34f-8ce67d9c31a8
scottda-batch-deployment-8-3-2020
scottda-model-deployment-8-3-2020
d435943c-d802-4e3b-b34c-e0f42fa4ee52


In [21]:
binding_uid = None

binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
if binding_uid is None:
    print("Adding binding:")
    binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance4ICP(wml_credentials=WML_CREDENTIALS))
    bindings_details = ai_client.data_mart.bindings.get_details()
else:
    print("Found binding:")
binding_uid

Found binding:


'405e817b-3715-453b-91a7-6a36186de087'

In [24]:
wml_models = wml_client.repository.get_model_details()
model_uid = None

for model_in in wml_models['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break
        
print(model_uid)

5b2016f1-ad4e-47f5-af86-32497f372383


# 8.0 Historical data <a name="historical"></a>

 ## 8.1 Insert historical payloads

The next section of the notebook downloads and writes historical data to the payload and measurement tables to simulate a production model that has been monitored and receiving regular traffic for the last seven days. This historical data can be viewed in the Watson OpenScale user interface. The code uses the Python and REST APIs to write this data.

In [11]:
!rm payload_history_*.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_0.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_1.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_2.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_3.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_4.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_5.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_6.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_7.json

rm: cannot remove ‘payload_history_*.json’: No such file or directory
--2020-08-14 20:20:26--  https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_0.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4834579 (4.6M) [text/plain]
Saving to: ‘payload_history_0.json’

100%[======================================>] 4,834,579   --.-K/s   in 0.09s   

2020-08-14 20:20:27 (53.8 MB/s) - ‘payload_history_0.json’ saved [4834579/4834579]

--2020-08-14 20:20:28--  https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/payload_history_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connec

In [14]:
!rm history_payloads_with_transaction_*.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_0.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_1.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_2.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_3.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_4.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_5.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_6.json


rm: cannot remove ‘history_payloads_with_transaction_*.json’: No such file or directory
--2020-08-14 20:23:17--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_0.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5209237 (5.0M) [text/plain]
Saving to: ‘history_payloads_with_transaction_id_0.json’

100%[======================================>] 5,209,237   --.-K/s   in 0.06s   

2020-08-14 20:23:17 (83.3 MB/s) - ‘history_payloads_with_transaction_id_0.json’ saved [5209237/5209237]

--2020-08-14 20:23:18--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/fastpath/history_payloads_with_transaction_id_1.json
Resol

In [15]:
from ibm_ai_openscale.utils.inject_demo_data import DemoData
import os

historicalData = DemoData(WOS_CREDENTIALS, ai_client)
historical_data_path=os.getcwd()

historicalData.load_historical_scoring_payload(subscription, deployment_uid,file_path=historical_data_path, day_template="history_payloads_with_transaction_id_{}.json" )

Loading historical scoring payload...
Day 0 injection.
Daily loading finished.
Day 1 injection.
Daily loading finished.
Day 2 injection.
Daily loading finished.
Day 3 injection.
Daily loading finished.
Day 4 injection.
Daily loading finished.
Day 5 injection.
Daily loading finished.
Day 6 injection.
Daily loading finished.


In [12]:
historyDays = 7

In [22]:
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
import datetime
import time

for day in range(historyDays):
    print('Loading day {}'.format(day + 1))
    history_file = 'payload_history_' + str(day) + '.json'
    with open(history_file) as f:
        payloads = json.load(f)
        hourly_records = int(len(payloads) / 24)
        index = 0
        for hour in range(24):
            recordsList = []
            for i in range(hourly_records):
                score_time = str(datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1))))
                recordsList.append(PayloadRecord(request=payloads[index]['request'], response=payloads[index]['response'], scoring_timestamp=score_time))
                index += 1
            subscription.payload_logging.store(records=recordsList)
print('Finished')


Loading day 1


ApiRequestFailure: Failure during payload logging storage. (POST https://zen1-cpd-zen1.aida-cpd3-dal10-b3c32x128-f2c6cdc6801be85fd188b09d006f13e3-0001.us-south.containers.appdomain.cloud/v1/data_marts/00000000-0000-0000-0000-000000000000/scoring_payloads)
Status code: 400, body: {"trace":"NDJiNGNhOGItNzgyNi00OTNhLThiMGMtNTA5MDRiZWVmZjAx","errors":[{"code":"AIQPO1007E","message":"Incoming payload does not fulfill requirements.  The field `request.fields` should correspond to the value specified in `subscription.entity.asset_properties.input_data_schema`. The issue corresponds to data_mart_id `00000000-0000-0000-0000-000000000000` and subscription_id `f978ce05-e4f6-42b0-81b2-be06884cd358` ","target":{"target_type":"field","name":"request.fields"},"parameters":["request.fields","subscription.entity.asset_properties.input_data_schema","00000000-0000-0000-0000-000000000000","f978ce05-e4f6-42b0-81b2-be06884cd358"]}]}

In [16]:
data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print(data_mart_id)

00000000-0000-0000-0000-000000000000


In [17]:
performance_metrics_url = WOS_CREDENTIALS['url'] + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

https://zen1-cpd-zen1.aida-cpd3-dal10-b3c32x128-f2c6cdc6801be85fd188b09d006f13e3-0001.us-south.containers.appdomain.cloud/v1/data_marts/00000000-0000-0000-0000-000000000000/metrics


## 8.2 Insert historical fairness metrics

In [ ]:
!rm fairness_history*
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/fairness_history.json

In [18]:
!rm history_fairness.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_fairness.json

rm: cannot remove ‘history_fairness.json’: No such file or directory
--2020-08-14 20:27:26--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_fairness.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1424078 (1.4M) [text/plain]
Saving to: ‘history_fairness.json’

100%[======================================>] 1,424,078   --.-K/s   in 0.03s   

2020-08-14 20:27:26 (39.5 MB/s) - ‘history_fairness.json’ saved [1424078/1424078]



In [19]:
import requests
from requests.auth import HTTPBasicAuth

def create_token():
    header = {
                    "Content-Type": "application/x-www-form-urlencoded",
                    "Accept": "application/json"
    }

    response = requests.Session().get(
            WOS_CREDENTIALS['url'] + '/v1/preauth/validateAuth',
            headers=header,
            auth=HTTPBasicAuth(
                WOS_CREDENTIALS['username'],
                WOS_CREDENTIALS['password']
            ),
            verify=False)

    response = handle_response(200, 'access token', response, True)
    token = response['accessToken']

    return token

In [25]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('history_fairness.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed
        
        qualityMetric = {
            'metric_type': 'fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': payloads[index]
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers, verify=False)

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## 8.3 Insert historical debias metrics

In [ ]:
!rm history_debias.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale//history_debias.json

In [26]:
!rm history_debias.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_debias.json

rm: cannot remove ‘history_debias.json’: No such file or directory
--2020-08-14 20:35:36--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_debias.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 565971 (553K) [text/plain]
Saving to: ‘history_debias.json’

100%[======================================>] 565,971     --.-K/s   in 0.02s   

2020-08-14 20:35:36 (25.5 MB/s) - ‘history_debias.json’ saved [565971/565971]



In [27]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('history_debias.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed

        qualityMetric = {
            'metric_type': 'debiased_fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': payloads[index]
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers, verify=False)
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## 8.4 Insert historical quality metrics

In [28]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.7,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.7
                    }
                ]
            }
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers, verify=False)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## 8.5 Insert historical confusion matrixes

In [ ]:
!rm history_quality_metrics.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/history_quality_metrics.json

In [29]:
!rm history_quality_metrics.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_quality_metrics.json

rm: cannot remove ‘history_quality_metrics.json’: No such file or directory
--2020-08-14 20:41:10--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_quality_metrics.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80099 (78K) [text/plain]
Saving to: ‘history_quality_metrics.json’

100%[======================================>] 80,099      --.-K/s   in 0.003s  

2020-08-14 20:41:10 (22.0 MB/s) - ‘history_quality_metrics.json’ saved [80099/80099]



In [30]:
measurements_url = WOS_CREDENTIALS['url'] + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/measurements'
print(measurements_url)

https://zen1-cpd-zen1.aida-cpd3-dal10-b3c32x128-f2c6cdc6801be85fd188b09d006f13e3-0001.us-south.containers.appdomain.cloud/v1/data_marts/00000000-0000-0000-0000-000000000000/measurements


In [31]:
with open('history_quality_metrics.json') as json_file:
    records = json.load(json_file)

for day in range(historyDays):
    index = 0
    measurments = []
    print('Day', day + 1)
    
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')

        measurement = {
            "monitor_definition_id": 'quality',
            "binding_id": subscription.binding_uid,
            "subscription_id": subscription.uid,
            "asset_id": subscription.source_uid,
            'metrics': [records[index]['metrics']],
            'sources': [records[index]['sources']],
            'timestamp': score_time
        }

        measurments.append(measurement)
        index+=1

    response = requests.post(measurements_url, json=measurments, headers=ai_client._get_headers(), verify=False)

print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## 8.6 Insert historical performance metrics

In [32]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(60, 600)
        score_resp = random.uniform(60, 300)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }

        response = requests.post(performance_metrics_url, json=[performanceMetric], headers=iam_headers, verify=False)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## 8.7 Insert historical manual labeling

In [33]:
manual_labeling_url = WOS_CREDENTIALS['url'] + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/manual_labelings'
print(manual_labeling_url)

https://zen1-cpd-zen1.aida-cpd3-dal10-b3c32x128-f2c6cdc6801be85fd188b09d006f13e3-0001.us-south.containers.appdomain.cloud/v1/data_marts/00000000-0000-0000-0000-000000000000/manual_labelings


In [ ]:
!rm history_manual_labeling.json
!wget https://raw.githubusercontent.com/scottdangelo/credit-risk-workshop-cpd/addData/data/openscale/history_manual_labeling.json

In [34]:
!rm history_manual_labeling.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_manual_labeling.json

rm: cannot remove ‘history_manual_labeling.json’: No such file or directory
--2020-08-14 20:44:16--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_manual_labeling.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 947956 (926K) [text/plain]
Saving to: ‘history_manual_labeling.json’

100%[======================================>] 947,956     --.-K/s   in 0.03s   

2020-08-14 20:44:16 (32.8 MB/s) - ‘history_manual_labeling.json’ saved [947956/947956]



In [35]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('history_manual_labeling.json', 'r') as history_file:
    records = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    record_json = []
    for hour in range(24):
        for record in records:
            if record['fastpath_history_day'] == day and record['fastpath_history_hour'] == hour:
                record['binding_id'] = binding_uid
                record['subscription_id'] = model_uid
                record['asset_revision'] = model_uid
                record['deployment_id'] = deployment_uid
                record['scoring_timestamp'] = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
                record_json.append(record)
    response = requests.post(manual_labeling_url, json=record_json, headers=iam_headers, verify=False)

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## 8.8 Additional data to help debugging

In [36]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Binding:', binding_uid)
# print('Scoring URL:', credit_risk_scoring_endpoint)

Datamart: 00000000-0000-0000-0000-000000000000
Model: 5b2016f1-ad4e-47f5-af86-32497f372383
Deployment: d435943c-d802-4e3b-b34c-e0f42fa4ee52
Binding: 405e817b-3715-453b-91a7-6a36186de087


## 8.9 Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [37]:
payload_data = subscription.payload_logging.get_table_content(limit=10)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability'])

,scoring_id,predictedLabel,probability
0,2592f241-862c-45d3-af50-6e9386283055-1,Risk,"[0.07410043332468508, 0.9258995666753149]"
1,2592f241-862c-45d3-af50-6e9386283055-2,No Risk,"[0.6586077076021792, 0.34139229239782076]"
2,2592f241-862c-45d3-af50-6e9386283055-3,No Risk,"[0.5003724887372748, 0.4996275112627253]"
3,2592f241-862c-45d3-af50-6e9386283055-4,Risk,"[0.10129360157081713, 0.898706398429183]"
4,2592f241-862c-45d3-af50-6e9386283055-5,Risk,"[0.20751677566806226, 0.7924832243319377]"
5,2592f241-862c-45d3-af50-6e9386283055-6,No Risk,"[0.5102158088353808, 0.48978419116461935]"
6,2592f241-862c-45d3-af50-6e9386283055-7,No Risk,"[0.5282956301309628, 0.4717043698690372]"
7,2592f241-862c-45d3-af50-6e9386283055-8,No Risk,"[0.7828985963639188, 0.2171014036360813]"
8,2592f241-862c-45d3-af50-6e9386283055-9,No Risk,"[0.720955885602349, 0.2790441143976511]"
9,2592f241-862c-45d3-af50-6e9386283055-10,No Risk,"[0.8448301477413261, 0.15516985225867388]"


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the OpenScale dashboard by going to the CPD `Home` page, and clicking `Services`. Choose the `OpenScale` tile and click the menu to `Open`. Click on the tile for the model you've created to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and can be deleted if necessary.

## Authors

Eric Martens, is a technical specialist having expertise in analysis and description of business processes, and their translation into functional and non-functional IT requirements. He acts as the interpreter between the worlds of IT and business.

Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Zilu (Peter) Tang, is a cognitive developer with experties in deep learning and enterprise AI solutions from Watson Openscale to many other cutting-edge IBM research projects.